In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install the GPU branch
%cd /content/drive/MyDrive
!git clone -b gpu https://github.com/maozdemir/privateGPT

/content/drive/MyDrive
fatal: destination path 'privateGPT' already exists and is not an empty directory.


In [3]:
# Install dependencies
%cd /content/drive/MyDrive/privateGPT
!pip3 install -r requirements.txt

!pip3 uninstall llama-cpp-python -y
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install llama-cpp-python

!pip3 install langchain

/content/drive/MyDrive/privateGPT
  Using cached llama_cpp_python-0.1.50-cp310-cp310-linux_x86_64.whl
  Attempting uninstall: llama-cpp-python
    Found existing installation: llama_cpp_python 0.1.65
    Uninstalling llama_cpp_python-0.1.65:
      Successfully uninstalled llama_cpp_python-0.1.65
Found existing installation: llama_cpp_python 0.1.50
Uninstalling llama_cpp_python-0.1.50:
  Successfully uninstalled llama_cpp_python-0.1.50
  Using cached llama_cpp_python-0.2.76-cp310-cp310-linux_x86_64.whl


In [4]:
# Install Koala 7B GGMLv3 Q4

!pip install llama-cpp-python==0.1.65 --force-reinstall --upgrade --no-cache-dir

!mkdir /content/drive/MyDrive/privateGPT/models/
%cd /content/drive/MyDrive/privateGPT/models/
!wget https://huggingface.co/TheBloke/koala-7B-GGML/resolve/main/koala-7B.ggmlv3.q4_0.bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 124.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 167.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.65-cp310-cp310-linux_x86_64.whl size=261612 sha256=ff69fcb54814e9558f2cc72f77daf8d05f6261ea50b70f83d0fedcf13da37105
  Stored in directory: /tmp/pip-ephem-wheel-cache-zykl8kw7/wheels/e0/62/84/21f820209ad725e813c2dd41eeda1a0dcb9184af7022d55c0d
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.0
    Uninstalling typing_extensions-4.12.0:
      Successfully uninstalled typing_extensions-4.12.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
 

In [5]:
# Modify .env
!mv /content/drive/MyDrive/privateGPT/example.env /content/drive/MyDrive/privateGPT/.env
!sed -i 's/n_gpu_layers=10/n_gpu_layers=500/' /content/drive/MyDrive/privateGPT/.env
!sed -i 's/MODEL_TYPE=GPT4All/MODEL_TYPE=LlamaCpp/' /content/drive/MyDrive/privateGPT/.env
!sed -i 's/ggml-gpt4all-j-v1.3-groovy.bin/koala-7B.ggmlv3.q4_0.bin/' /content/drive/MyDrive/privateGPT/.env
!sed -i 's/IS_GPU_ENABLED=False/IS_GPU_ENABLED=True/' /content/drive/MyDrive/privateGPT/.env

mv: cannot stat '/content/drive/MyDrive/privateGPT/example.env': No such file or directory


In [1]:
# These were required before (for some filetypes), idk now
!apt install pandoc
!pip3 install unstructured

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
# Ingest sources
%cd  /content/drive/MyDrive/privateGPT/
!python3 ingest.py

/content/drive/MyDrive/privateGPT
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Appending to existing vectorstore at db
Using embedded DuckDB with persistence: data will be stored in: db
Loading documents from source_documents
Loading new documents: 0it [00:00, ?it/s]
No new documents to load


In [4]:
# Run the chatbot
%cd /content/drive/MyDrive/privateGPT/
!python3 privateGPT.py

/content/drive/MyDrive/privateGPT
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Using embedded DuckDB with persistence: data will be stored in: db
llama.cpp: loading model from models/koala-7B.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 1000
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B

In [ ]:
# Clear database
!rm -rf /content/privateGPT/db

In [ ]:
# Zip the database if will reuse without recreating vectors
!zip -r /content/db.zip /content/privateGPT/db